In [23]:
import os
import warnings
warnings.filterwarnings("ignore")

### base directory

In [24]:
base_dir = 'people'

### Making Folder for each train and test

In [25]:
import os
import shutil
from sklearn.model_selection import train_test_split

def split_images_by_gender(input_folder, output_folder, train_ratio=0.8):
    # Define gender categories
    categories = ['MEN', 'WOMAN']

    for category in categories:
        category_path = os.path.join(input_folder, category)
        if not os.path.exists(category_path):
            print(f"Directory {category_path} does not exist.")
            continue

        images = [f for f in os.listdir(category_path) if os.path.isfile(os.path.join(category_path, f))]
        
        # Split images into train and test sets
        train_images, test_images = train_test_split(images, train_size=train_ratio, random_state=42)
        
        # Define train and test paths
        train_path = os.path.join(output_folder, 'train', category)
        test_path = os.path.join(output_folder, 'test', category)
        
        # Create directories if they don't exist
        os.makedirs(train_path, exist_ok=True)
        os.makedirs(test_path, exist_ok=True)
        
        # Copy images to train folder
        for img in train_images:
            shutil.copy(os.path.join(category_path, img), os.path.join(train_path, img))
        
        # Copy images to test folder
        for img in test_images:
            shutil.copy(os.path.join(category_path, img), os.path.join(test_path, img))

# Define paths
input_folder = 'dataset'
output_folder = 'people'

# Split images
split_images_by_gender(input_folder, output_folder, train_ratio=0.8)


In [41]:
men_dir = os.path.join(base_dir, 'train/MEN')
women_dir = os.path.join(base_dir, 'train/WOMAN')

men_test_dir = os.path.join(base_dir, 'test/MEN')
women_test_dir = os.path.join(base_dir, 'test/WOMAN')


### import modules

In [42]:
import tensorflow
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Input, layers, models


### train data with data augmentation

In [43]:
train_datagen = ImageDataGenerator(
    
    rescale = 1./255,
    rotation_range = 40,
    horizontal_flip = True
)

test_datagen = ImageDataGenerator(rescale=1./255)

In [49]:
train_generator = train_datagen.flow_from_directory(
    
    women_dir men_dir
    target_size = (28,28),
    batch_size = 20,
    class_mode = 'binary'
    
)

valid_generator = test_datagen.flow_from_directory(
    
    women_test_dir,men_test_dir
    target_size = (28,28),
    batch_size = 20,
    class_mode = 'binary'
)

SyntaxError: invalid syntax. Perhaps you forgot a comma? (1869238458.py, line 3)

### Apply model 

In [ ]:
model = models.Sequential([
    Input(shape=(28, 28, 3)),
    layers.Conv2D(32,(3, 3), activation= 'relu'),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(32,(3, 3), activation= 'relu'),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid') #output layer 
])

### summary of complete data

In [31]:
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)               │ (None, 26, 26, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 11, 11, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 5, 5, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 800)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 512)            │       410,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 486,049 (1.85 MB)

 Trainable params: 486,049 (1.85 MB)

 Non-trainable params: 0 (0.00 B)

### complie the model

In [32]:
model.compile(
    
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

### train the model

In [33]:
model.fit(
     
    train_generator,
    epochs=10,
    validation_data=valid_generator
)

Epoch 1/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 24s 352ms/step - accuracy: 0.5209 - loss: 0.6994 - val_accuracy: 0.5423 - val_loss: 0.6829
Epoch 2/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 20s 350ms/step - accuracy: 0.5919 - loss: 0.6715 - val_accuracy: 0.6269 - val_loss: 0.6409
Epoch 3/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 21s 361ms/step - accuracy: 0.6149 - loss: 0.6461 - val_accuracy: 0.6423 - val_loss: 0.6184
Epoch 4/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 20s 353ms/step - accuracy: 0.6676 - loss: 0.5825 - val_accuracy: 0.6462 - val_loss: 0.6157
Epoch 5/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 21s 360ms/step - accuracy: 0.6775 - loss: 0.5844 - val_accuracy: 0.7000 - val_loss: 0.5718
Epoch 6/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 20s 352ms/step - accuracy: 0.7261 - loss: 0.5488 - val_accuracy: 0.6577 - val_loss: 0.6089
Epoch 7/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 22s 367ms/step - accuracy: 0.6802 - loss: 0.5842 - val_accuracy: 0.6654 - val_loss: 0.5868
Epoch 8/10
52/52 ━━━━━━━━━━━━━━━━━━━━ 21s 356ms/step - accuracy: 0.6999 - loss: 0.5353 - val_accu

In [34]:
test_loss, test_acc = model.evaluate(valid_generator)
print(f'Test accuracy: {test_acc}')

13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 263ms/step - accuracy: 0.7322 - loss: 0.5466
Test accuracy: 0.7153846025466919


### Almost Accuracy using CNN is 71%

In [35]:
model.save('your_data.h5')

### Now Fature Extraction transfer learning

In [14]:
import tensorflow
from keras import Sequential
from keras.layers import Flatten, Dense
from keras.applications.vgg16 import VGG16

### freeze CNN layers

In [15]:
conv_base = VGG16(

    include_top = False,
    input_shape = (150,150,3)
)

### Set own size od dense layers

In [16]:
model = Sequential()
model.add(conv_base),
model.add(Flatten()),
model.add(Dense(512, activation= 'relu')),
model.add(Dense(1, activation= 'sigmoid'))

In [17]:
conv_base.trainable = False

In [18]:
from tensorflow.keras.preprocessing.image import  ImageDataGenerator

### Train data with augmentation

In [19]:
batch_size = 32

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        'people/train',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        'people/test',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')

Found 1031 images belonging to 2 classes.
Found 260 images belonging to 2 classes.


#### compile the model

In [20]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

#### Start training of model

In [21]:
history = model.fit(
        train_generator,
        epochs=5,
        validation_data=validation_generator)

Epoch 1/5
33/33 ━━━━━━━━━━━━━━━━━━━━ 300s 9s/step - accuracy: 0.6986 - loss: 0.9928 - val_accuracy: 0.8692 - val_loss: 0.3050
Epoch 2/5
33/33 ━━━━━━━━━━━━━━━━━━━━ 288s 9s/step - accuracy: 0.9170 - loss: 0.1992 - val_accuracy: 0.7923 - val_loss: 0.5653
Epoch 3/5
33/33 ━━━━━━━━━━━━━━━━━━━━ 292s 9s/step - accuracy: 0.8889 - loss: 0.2398 - val_accuracy: 0.8731 - val_loss: 0.3155
Epoch 4/5
33/33 ━━━━━━━━━━━━━━━━━━━━ 291s 9s/step - accuracy: 0.9561 - loss: 0.1345 - val_accuracy: 0.8885 - val_loss: 0.3125
Epoch 5/5
33/33 ━━━━━━━━━━━━━━━━━━━━ 286s 9s/step - accuracy: 0.9535 - loss: 0.1076 - val_accuracy: 0.8577 - val_loss: 0.3334


### Using Feature Extraction Transfer learning Accuracy 87%
### using 5 epochs